### In this File I'm going to explore Data from DrugCombDB

Some theory Background:<br>
DrugCombDB combines data from 4 kinds of sources:<br>
1. HTS assays:                                       Main component; Dose-response landscapes from combinational therapy
2. AstraZeneca Dataset:                        They did a ML competition on synergistic rules. Is integrated here
3. Manual Literature curations            They reviewed 10k articles on PubMed and integrated data from there
4. Other DBs                                           They added data from DCDB, DrugCentral, TTD, ASDC & DrugBank


They found data on 3+ combinations, which is rare. Added a small extra dataset.

First I will try to connect with the API, I might later to prefer to download datasets

In [5]:
import requests
import json

url = 'http://drugcombdb.denglab.org:8888/'
data = {
    'combination_id': '444439',
}

r = requests.post(url, data=data)
print(r.text)

{"timestamp":"2022-02-10T18:16:55.947+0000","status":404,"error":"Not Found","message":"No message available","path":"/"}


In [2]:
import requests

ApiConfig = {
  "baseURL": "http://drugcombdb.denglab.org:8888",
  "isCompleteResponse": False,
  "timeout": 10000,
}

def getDrugCombPagination(page, size):
  return requests.get(
    url=ApiConfig["baseURL"] + "/combination/list",
    params={"page": page, "size": size},
    timeout=ApiConfig["timeout"],
  )

def run():
  print(getDrugCombPagination(1, 5).json())

run()

{'code': 200, 'data': {'total': 91983, 'page': [{'blockId': 1, 'drugRowName': '5-FU', 'drugColName': 'ABT-888', 'concRow': 0.0, 'concCol': 0.0, 'concRowUnit': 'uM', 'concColUnit': 'uM', 'response': 100.00062645381074, 'blissScore': 0.0, 'source': 'ONEIL'}, {'blockId': 1, 'drugRowName': '5-FU', 'drugColName': 'ABT-888', 'concRow': 0.0, 'concCol': 0.35, 'concRowUnit': 'uM', 'concColUnit': 'uM', 'response': 101.99300946838412, 'blissScore': 0.0, 'source': 'ONEIL'}, {'blockId': 1, 'drugRowName': '5-FU', 'drugColName': 'ABT-888', 'concRow': 0.0, 'concCol': 1.08, 'concRowUnit': 'uM', 'concColUnit': 'uM', 'response': 100.95767331883155, 'blissScore': 0.0, 'source': 'ONEIL'}, {'blockId': 1, 'drugRowName': '5-FU', 'drugColName': 'ABT-888', 'concRow': 0.0, 'concCol': 3.25, 'concRowUnit': 'uM', 'concColUnit': 'uM', 'response': 99.93037198360902, 'blissScore': 0.0, 'source': 'ONEIL'}, {'blockId': 1, 'drugRowName': '5-FU', 'drugColName': 'ABT-888', 'concRow': 0.0, 'concCol': 10.0, 'concRowUnit': 'u

In [3]:
#do more different requests later

added REGR to the file name to indicate, that the data in der are numeric <br><br>

Combinations of drugs can have synergistic or antagonistic effects on each other, compared to single use<br>
the dataset "drugcombs_scored" includes 4 different synergy metrics, in descending relevance.<br>
The last one is a novel metric, espcially for HTC data:<br>
1. Loewe additivity
2. Bliss independence Models (ratio of change of effect due to combination)
3. HSA (comparing difference of max effect)
4. zero interaction potency (ZIP)


### ADD FIGURE AND DESCRIPTON FROM PAPER

In [6]:
import pandas as pd
scoredComb = pd.read_csv("drugCombDBdata/synergy-score-data/REGRdrugcombs_scored.csv")
print(len(scoredComb))
print(scoredComb.head(3))
print("----------------------------------------------------------------")
print(scoredComb[-3:])

498865
   ID Drug1    Drug2 Cell line   ZIP  Bliss  Loewe    HSA
0   1  5-FU  ABT-888     A2058  1.72   6.26  -2.75   5.54
1   2  5-FU  ABT-888     A2058  5.88  12.33   3.33  11.61
2   3  5-FU  ABT-888     A2058  3.59  11.66   2.65  10.94
----------------------------------------------------------------
            ID           Drug1                      Drug2 Cell line     ZIP  \
498862  498863       Erlotinib  Valproic acid sodium salt    DIPG25 -15.768   
498863  498864  Bafilomycin A1  Valproic acid sodium salt    DIPG25   2.208   
498864  498865     Mebendazole  Valproic acid sodium salt    DIPG25   6.807   

         Bliss   Loewe    HSA  
498862 -15.776 -25.465 -6.212  
498863  10.567  -9.455  1.525  
498864  11.948 -18.311  1.149  


In [1]:
import pandas as pd
pd.read_csv("./drugCombDBdata/synergy-score-data/SHORTdrugcombs_response.csv").head()


,BlockID,Row,Col,DrugRow,DrugCol,ConcRow,ConcCol,Response,ConcRowUnit,ConcColUnit,source
0,1,1,1,5-FU,ABT-888,0.0,0.00,100.000626,uM,uM,ONEIL
1,1,1,2,5-FU,ABT-888,0.0,0.35,101.993009,uM,uM,ONEIL
2,1,1,3,5-FU,ABT-888,0.0,1.08,100.957673,uM,uM,ONEIL
3,1,1,4,5-FU,ABT-888,0.0,3.25,99.930372,uM,uM,ONEIL
4,1,1,5,5-FU,ABT-888,0.0,10.00,98.861202,uM,uM,ONEIL


In [2]:
import pandas as pd
from rdkit import Chem
data3combs = pd.read_csv("./drugCombDBdata/ThreeDrugCombs.csv")
data3combs.head()

,Drug1,Drug2,Drug3,Drug1 Concentration,Drug2 Concentration,Drug3 Concentration,Small Mol Conc Unit,Replicate,Total Number of Cells (Nuclei),Number of Apoptotic Cells,Relative Viability,Mean Relative Viability,Apoptosis Fraction,Mean Apoptosis Fraction,Cell Name
0,Vemurafenib,Trametinib,Dasatinib,0.0,0.0,0.000,uM,1,15785,755,0.9659,1.0000,0.0478,0.0508,A-375
1,Vemurafenib,Trametinib,Dasatinib,0.0,0.0,0.000,uM,2,15978,849,0.9722,1.0000,0.0531,0.0508,A-375
2,Vemurafenib,Trametinib,Dasatinib,0.0,0.0,0.000,uM,3,17422,897,1.0619,1.0000,0.0515,0.0508,A-375
3,Vemurafenib,Trametinib,Dasatinib,0.0,0.0,0.316,uM,1,3999,254,0.2407,0.2809,0.0635,0.0601,A-375
4,Vemurafenib,Trametinib,Dasatinib,0.0,0.0,0.316,uM,2,4717,279,0.2852,0.2809,0.0591,0.0601,A-375


For the classification task:
Just make features out of drugs, chemistry, cell lines etc and build a model for classification
...or build regression task

In [3]:
#don't unneccessarily execute, its big!
drugProteinLinks = pd.read_csv("./drugCombDBdata/descriptions/drug_protein_links.tsv",sep="\t")
print("number of Drug-Protein combinations:",len(drugProteinLinks))
drugProteinLinks.head(3)

number of Drug-Protein combinations: 15473939


,chemical,protein,experimental,prediction,database,textmining,combined_score
0,CIDm91758680,9606.ENSP00000257254,0,0,0,278,279
1,CIDm91758680,9606.ENSP00000302120,0,0,0,154,154
2,CIDm91758408,9606.ENSP00000006777,0,0,0,225,225


In [3]:
def sample (items, n): result = [] for i, item in enumerate (items):
    if i < n:
    result.append(item)
    else:
    j = randint(0, i) # random integer, inclusive
    if j < n:
    # (Fill in the missing line here)
    result[j] = item
    return result

0 [1]


In [7]:
import queue
import threading
import time
q = queue.Queue ()
for i in [3,2,1]:
    def f():
        time.sleep(i)
        q.put(i)
    threading.Thread (target=f).start()
print (q.get())

1


In [1]:
lst = [1,3,4]
for i in enumerate(lst):
    print(i)

(0, 1)
(1, 3)
(2, 4)


In [ ]:
#more analysis in colab
del drugProteinLinks

In [3]:
#don't unneccessarily execute, its big!
proteinProteinLinks = pd.read_csv("./drugCombDBdata/descriptions/protein_protein_links.txt",sep=" ")
print("Length:",len(proteinProteinLinks))
proteinProteinLinks.head(3)

Length: 11759454


,protein1,protein2,combined_score
0,9606.ENSP00000000233,9606.ENSP00000272298,490
1,9606.ENSP00000000233,9606.ENSP00000253401,198
2,9606.ENSP00000000233,9606.ENSP00000401445,159


In [25]:
#more in colab
del proteinProteinLinks

In [27]:
#wondering if they've cut out something
with open("./drugCombDBdata/descriptions/protein_protein_links.txt") as myfile:
    head = [next(myfile) for x in range(5)]
print(head)
#no they didn't

['protein1 protein2 combined_score\n', '9606.ENSP00000000233 9606.ENSP00000272298 490\n', '9606.ENSP00000000233 9606.ENSP00000253401 198\n', '9606.ENSP00000000233 9606.ENSP00000401445 159\n', '9606.ENSP00000000233 9606.ENSP00000418915 606\n']


In [12]:
drugChem = pd.read_csv("./drugCombDBdata/descriptions/drug_chemical_info.csv")
print("Numer of Drugs:",len(drugChem))
drugChem.head(3)

Numer of Drugs: 3059


,drugName,cIds,drugNameOfficial,molecularWeight,smilesString
0,Bendamustine,CIDs00065628,bendamustine,358.26284,CN1C2=C(C=C(C=C2)N(CCCl)CCCl)N=C1CCCC(=O)O
1,Lonidamine,CIDs00039562,lonidamine,321.15810,C1=CC=C2C(=C1)C(=NN2CC3=C(C=C(C=C3)Cl)Cl)C(=O)O
2,Lenalidomide,CIDs00216326,lenalidomide,259.26062,C1CC(=O)NC(=O)C1N2CC3=C(C2=O)C=CC=C3N


In [13]:
cellLines = pd.read_csv("./drugCombDBdata/descriptions/cell_Line.csv")
print("Number of Cell Lines:",len(cellLines))
cellLines.head(3)

Number of Cell Lines: 104


,cellName,cosmicId,tag
0,786-0,905947.0,NaN
1,A498,905948.0,NaN
2,A549/ATCC,905949.0,NaN


Making simple univariate regression model here on HSA:

In [43]:
temp=drugChem.loc[:,["drugName","cIds","smilesString"]].set_index("drugName")

smilesHSAtable = scoredComb.join(temp,on="Drug1",how="left").loc[:,["cIds","smilesString","HSA"]]
del temp
smilesHSAtable.head(3)

,cIds,smilesString,HSA
0,CIDs00003385,C1=C(C(=O)NC(=O)N1)F,5.54
1,CIDs00003385,C1=C(C(=O)NC(=O)N1)F,11.61
2,CIDs00003385,C1=C(C(=O)NC(=O)N1)F,10.94


add molecules to the table

In [77]:
smilesHSAtable[smilesHSAtable.loc[:,"smilesString"].astype(str)=="nan"]

,cIds,smilesString,HSA
208,NaN,NaN,1.360
209,NaN,NaN,0.910
210,NaN,NaN,1.720
211,NaN,NaN,6.110
212,NaN,NaN,2.220
...,...,...,...
498834,CIDs00060838,NaN,-2.636
498837,CIDs118958122,NaN,-1.359
498843,CIDs09852573,NaN,5.889
498851,CIDs118191391,NaN,-1.262


In [ ]:
Chem.MolFromSmiles(smilesHSAtable.loc[0,"smilesString"])
counter=0
for x in smilesHSAtable.loc[:,"smilesString"].astype(str):
    if counter==0:
        tem=Chem.MolFromSmiles(x)

In [64]:
# making a mol list from the smiles
from mediumArticleSolubility.mediumArticleFunctionGenerate import generate

smilesHSAtable['mols'] = [Chem.MolFromSmiles(x), counter+=1 for x in smilesHSAtable.loc[:,"smilesString"]]
smilesHSAtable


descriptorDF = generate(smilesHSAtable.loc[:,"smilesString"]) #don't rely on indices, they change in any filtering step
descriptorDF.head(3)

SyntaxError: invalid syntax (<ipython-input-64-1ea043593837>, line 4)